In [1]:
# system packages
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as mtplt
import matplotlib.gridspec as gridspec
%matplotlib qt

import scipy.linalg as scLinAlg
import scipy.signal as sigP


In [2]:
# individual packages
import sg, sa, sp, obq

mtplt.close('all')

sNbins = 2**11
sFs = sNbins
sT = 1 / sFs
sL = 83
sL_h = math.floor(sL/2)
sRecFilterFrequ = 132


Generate input signal

In [3]:
### Signal generation ###
v_n = np.arange(sNbins).reshape(-1, 1)
vxFrequ = (np.arange(1, 131, step=3) * sFs / sNbins).reshape(-1, 1)
vxPhaseInit = np.random.rand(len(vxFrequ), 1) * 2 * np.pi

vx, vTime = sg.signalGen(v_n, vxFrequ, vxPhaseInit, sFs, 'real')
vx = sg.MFnormalize(vx, np.array([-1, 1]))

Generate FiltersCoeff and corresp. Matrices

In [4]:
### Generate ideal matrices ###
vRIdeal = sp.idealBinFilt(sNbins, sg.freq2Bin(sRecFilterFrequ, sNbins, sFs), 'normal')
mRIdeal = scLinAlg.toeplitz(vRIdeal)

vFilt = sigP.firwin(sL, sg.freq2digFc(sRecFilterFrequ,sFs), window = "hann")
# Replace zeros with eps
vFilt[vFilt == 0] = np.finfo(float).eps
mFilt = sp.convmtx(vFilt,sNbins,'colWise')
mFilt = mFilt[sL_h-1:(sL_h+sNbins-1),:]
mFilt = np.tril(mFilt)

Quantize the input signal

In [5]:
vC = np.zeros((sNbins,1), dtype = 'float')
vBSequSingle, ve = obq.iterSequQ(vx,mFilt,vC,0)

Frequency Analysis

In [6]:
vBfft = np.fft.fft(vBSequSingle)
vBfftMag = 20*sa.safelog10(np.abs(vBfft) / np.max(abs(vBfft))) 

vX = np.fft.fft(vx)
vXMag = 20*sa.safelog10(np.abs(vX) / np.max(abs(vX)))
# Frequency bins
vFreq = np.fft.fftfreq(sNbins, sT)

SNR Calculations

In [11]:
sVX_MSE, sVX_SNRdb, sVX_PSNRdb = sa.evalN(mRIdeal @ (vx-vx), mRIdeal @ vx)
sVB_MSE, sVB_SNRdb, sVB_PSNRdb = sa.evalN(mRIdeal @ (vx-vBSequSingle), mRIdeal @ vx)

Plots

In [12]:
###### PLOTTING ######
figOne = mtplt.figure()
Pltgs = gridspec.GridSpec(3, 2)

pltDiscTime = figOne.add_subplot(Pltgs[0,:])
pltDiscTime.plot(vx)
pltDiscTime.set_title('Input Signal SNR: {snr} dB'.format(snr = round(sVX_SNRdb,2)))
pltDiscTime.set_xlabel('Samples $n$', fontsize = 11)
pltDiscTime.set_ylabel('Amplitude', fontsize = 11)
pltDiscTime.set_xlim([0,sNbins])
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

pltObsOne = figOne.add_subplot(Pltgs[1,0])
pltObsOne.plot(vBSequSingle)
pltObsOne.set_title('Single-Sequential Quantization')
pltObsOne.set_xlabel('Samples $n$', fontsize = 11)
pltObsOne.set_ylabel('Amplitude', fontsize = 11)
pltObsOne.set_xlim([0,sNbins])
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

pltObsTwo = figOne.add_subplot(Pltgs[1,1])
pltObsTwo.set_title('Block-Sequential Structure')
pltObsTwo.set_xlabel('Samples $n$', fontsize = 11)
pltObsTwo.set_ylabel('Amplitude', fontsize = 11)
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

pltFreqOne = figOne.add_subplot(Pltgs[2,0])
pltFreqOne.plot(vFreq[:sNbins // 2], vBfftMag[:sNbins // 2])
pltFreqOne.set_title('Frequency Spectrum One  SNR: {snr} dB'.format(snr = round(sVB_SNRdb,2)))
pltFreqOne.set_xlabel('Samples $n$', fontsize = 11)
pltFreqOne.set_ylabel('Amplitude', fontsize = 11)
pltFreqOne.set_xlim([0,sNbins/2])
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

pltFreqTwo = figOne.add_subplot(Pltgs[2,1])
pltFreqTwo.set_title('Frequency Spectrum Two')
pltFreqTwo.set_xlabel('Samples $n$', fontsize = 11)
pltFreqTwo.set_ylabel('Amplitude', fontsize = 11)
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

mtplt.tight_layout()
mtplt.show()